# Spherical CNNs


1. Какую задачу решают авторы?

    Авторы в статье вводят обобщение сверточных сетей (CNN) для сферических сигналов: трехмерных данных, обладающих симметрией относительно трехмерных вращений. Примерами таких данных могут быть данные камер кругового обзора, распознавание моделей 3D объектов, регрессионные задачи в молекулярном моделировании и другие. Непосредственное применение двумерных сверточных моделей на сферических данных, при проецировании их на плоскость, неэффективно, так как при таком проектировании нарушается присущая данным симметрия относительно трехмерных вращений. Для решения этой задачи авторы вводят свертки (а, точнее, кросс-корреляции) на сфере ($S^2$) и на группе вращений в 3D ($\text{SO}(3)$), предлагают вычислительно эффективные и устойчивые алгоритмы для их нахождения. Использование введенных слоев в составе нейросетей позволяет сохранять вращательные симметрии входных данных между слоями сети, что позволяет создавать компактные универсальные модели для анализа трехмерных данных, эффективно извлекающие признаки, обладающие симметрией относительно вращений.


2. Какова основная идея предлагаемого авторами решения поставленной задачи?  
    
    Центральной идеей работы является обобщение концепции двумерных сверток на плоскости на сферические сигналы, заменяя симметрию относительно сдвигов на симметрию относительно вращений в пространстве. Для сферических функций, сферического сигнала $f : S^2 \rightarrow \mathbb{R}^K$ и фильтра $\psi : S^2 \rightarrow \mathbb{R}^K$ ($K$ - число каналов), сферическая кросс-корреляция (**Spherical Correlation**) вводится следующим образом:

    \begin{equation*}
    [\psi \star f](R) = \langle L_R \psi, f \rangle = \int_{S^2} \sum_{k=1}^K \psi_k(R^{-1} x) f_k(x) dx,
    \end{equation*}

    где $R\in \text{SO}(3)$, результатом вычисления является также функция на $\text{SO}(3)$, $dx = d\alpha \sin(\beta) d\beta /4\pi$ в сферических координатах. Данное определение гарантирует эквивариантность относительно вращений: если входные данные будут повернуты, то соответствующим образом будут будет повернута и полученные на выходе признаки. Авторы отмечают, что в классическом определении сферической кросс-корреляции рассматривают отображение в сферические функции, что автоматически накладывает условие симметричности фильтра относительно оси $z$, сильно снижая обобщающие способности нейросети. 
    
    В последующих слоях сети, в которых выходами являются функции на $\text{SO}(3)$, предлагается использовать кросс-корреляции функций $f, \psi : \text{SO}(3) \rightarrow \mathbb{R}^K$ (**Rotation Group Correlation**), определяемые как:

    \begin{equation*}
    [\psi \star f](R) = \langle L_R \psi, f \rangle = \int_{\text{SO}(3)} \sum_{k=1}^K \psi_k(R^{-1} Q) f_k(Q) dQ,
    \end{equation*}

    где $R\in \text{SO}(3)$, результатом вычисления является также функция на $\text{SO}(3)$, $dQ = d\alpha \sin(\beta) d\beta d\gamma / (8 \pi^2)$ в ZYZ углах Эйлера. Авторами показано, что данные преобразования действительно эквивариантны относительно вращений. 
    
    Первый слов spherical correlation ($S^2$-conv) и слои rotation group correlation ($\text{SO}(3)$-conv)? между которыми расположены нелинейности (и батч-нормализация), вместе с специфичной для задачи головой являются Spherical CNN, которые ввели в своей статье авторы.

    Кроме этого, предлагается вычислительно устойчивый и эффективный алгоритм вычисления spherical correlation и rotation group correlation на основе быстрого обобщенного преобразования Фурье (GFFT). Таким образом, использование сверточных сетей на основе spherical correlation и rotation group correlation позволяют эффективно обрабатывать данные, обладающие вращательной симметрией, не прибегая к соответствующей аугментации данных и дополнительному feature engineering'у. 
   

3. Каковы результаты, полученные авторами? 
    
    Авторы математически доказали свойства эквивариантности относительно вращений непрерывных преобразований spherical correlation и rotation group correlation. Был также проведен ряд вычислительных экспериментов, первый из которых демонстрирует применимость дискретизованной версии описанных операций. Далее на ряде тестовых задач демонстрируется универсальность и эффективность нейросетей, построенных на основе spherical correlation и rotation group correlation (Spherical CNNs). Список экспериментов следующий:

    * Экспериментально было показано, что дискретизированные версии spherical correlation и rotation group correlation сохраняют эквивариантность, доказанную для непрерывных преобразований, в пределах допустимых погрешностей даже для более глубоких сетей.
    * На датасете Rotated MNIST Spherical CNNs демонстирует сравнимые с классическими CNN показатели точности классификации на цифрах без вращения (0.96 и 0.98 соот.). Однако при наличии вращений в датасете точность классификации классической модели CNN значительно падает (0.23), особенно сильное ухудшение точности происходит при отсутсвии в обучающей выборки цифр с преобразованиями вращения (0.11). В свою очередь, Spherical CNNs в случае наличия вращений в тестовых данных, даже без вращений в обучающей выборке, практически не теряет в качестве классификации (0.94, при налчии вращений на трейне - 0.95).
    * Классификация 3D объектов (SHREC17): Spherical CNNs достигает высоких результатов, 2-3 мест среди SoTA моделей (на момент написания статьи) по ключевым метрикам (P@N: 0.701 (3 место); R@N 0.711 (2 место); F1@N: 0.699 (3 место); mAP: 0.676 (2 место), nDCG (2 место)). При этом лучшие SoTA модели специализированы для решения описанной задачи, тогда как Spherical CNNs является универсальной моделью для данных с симметриями вращения.
    * Molecular Energy Regression (QM7): в задаче предсказания энергии атомизации молекул Spherical CNN достигла RMSE = 8.47, опережая модели на основе ядерных методов и MLP с сортированными или случайными перестановками матрицы Кулона. При этом единственная модель, превосходящая Spherical CNN по RMSE, масштабируется экспоненциально с ростом числа атомов (как число их перестановок), что снижает ее применимость для анализа больших молекул.
    

1. В чём заключается смысл понятия эквивариантности? Являются ли обычные свёртки эквивариантными относительно каких-либо преобразований?  

    Эквивариантность &mdash; это свойство функции или отображения согласованно изменяться при преобразованиях входных данных, связанных с определённой симметрией (например, сдвигом, вращением или масштабированием). Если преобразование входных данных приводит к предсказуемому и согласованному с ним изменению выходных данных, то такое отображение называется эквивариантным относительно данного преобразования. 

    Пусть $G$ &mdash; группа преобразований (например, вращений), а $T_g: X \rightarrow X$ &mdash; действие элемента $g\in G$ на $X$. Отображение $f: X \rightarrow Y$ называется эквивариантным относительно $G$, если существует действие $T_g': Y \rightarrow Y$ элемента той же группы на $Y$ такое, что:

    $ f(T_h(x)) =  T_g'(f(x)),\quad \forall g\in G,\; x\in X.$

    В машинном обучении эквивариантность позволяет создавать модели, которые учитывают симметрии данных, улучшая обобщающие способности модели (например, $G$-эквивариантные нейросети, в данной статье &mdash; $\text{SO}(3)$-эквивариантные нейросети).


2. Зачем нужны и чем отличаются Spherical Correlation и Rotation Group Correlation?  

    Определения Spherical Correlation и Rotation Group Correlation были даны выше. Данные отображения являются эквивариантными относительно вращений в трехмерном пространстве. Spherical correlation определяется на сфере $S^2$, свертка проводится с фильтром, над которым сделали вращение, выходом является элемент $\text{SO}(3)$. Данное отображение используется на входных слоях. Rotation Group Correlation &mdash; преобразование $\text{SO}(3)$, используется уже с данными, обработанными Spherical Correlation на более глубоких слоях.


3. Как организовать предсказание числовых значений или логитов для задачи классификации или регрессии, если внутри сети используются сферические свёртки?  

    Когда в сети используются сферические свертки, скрытыми представлениями данных являются функции, определенные на $S^2$ или $\text{SO}(3)$. Для задачи классификации или регрессии данные прелставления должны быть агрегированы, например, используя пулинги и полносвязные слои, чтобы получить эквивариантное к вращениям векторное представление входных данных. Далее данное векторное представление может быть использовано для решения конкретной задачи, используя необходмую для этого голову нейросети. Так, в задаче классификации 3D объектов после серии из $S^2$conv-BN-ReLU и ряда  $\text{SO}(3)$conv-BN-ReLU использовался max-pooling, BN и линейный слой, на выходе которого были логиты для использования в softmax.


4. Объясните своими словами, как авторы предлагают эффективно вычислять Spherical Correlation или Rotation Group Correlation (достаточно описать один из методов)?  

    Авторы предлагают эффективно вычислять Spherical Correlation и Rotation Group Correlation, преобразуя сигнал и фильтр в спектральную область при помощи обобщенного преобразования Фурье (GFT). Для $S^2$, предлагается проецировать сигнал на базис сферических гармоник $Y_{m}^l$, для $\text{SO}(3)$ &mdash; на D-функции Вигнера $D_{mn}^l$. В соответствии с теоремой о свертке, свертка функций является произведением Фурье-образов функций, данный результат обобщается на обобщенное преобразование Фурье. Для GFT существуют алгоритмы быстрого вычисления (GFFT), аналогичные FFT, позволяющие значительно снизить сложность относительно наивного алгоритма (для $\text{SO}(3)$ наивный алгоритм имеет сложность $O(n^6)$).


5. Как авторы проверяли эффективность предложенной модели, и какие результаты были получены?

    Авторы экспериментально проверили, что при дискретизации модели и сигналов ошибки в свойтсвах экваивариантности. При росте числа слоев и разрешения точность эквивариантности в смысле центрированного RNMSE остается приемлемой. Далее авторы проверяют универсальность свой модели для вращательно-симметричных данных на ряде задач классификации и регрессии. Модель дествительно выучивает эквивариантные к вращению векторые представления данных, демонстирует сравнимое c классическими SoTA решениями качество работы, при этом оставаясь компактной и универсальной моделью. Численные результаты и частные для каждого из экспериментов выводы довольно подробно описаны в п.3 первой части.